因为一些应用可能需要动态创建和执行Datalog语句（比如实时），这时就可以使用这些指令。下面的例子给出了所有动态Datalog语句的接口：

In [1]:
from pyDatalog.pyDatalog import assert_fact, retract_fact, load, ask
from pyDatalog import pyDatalog

# load(string): 从字符中加载Datalog语句
load("""
    + parent('Kangxi', 'Yongzheng')
    + parent('Yongzheng', 'Qianlong')
    ancestor(X,Y) <= parent(X,Y)
    ancestor(X,Y) <= parent(X,Z) & ancestor(Z,Y)
    """)
# assert_fact(string,不限量任意参数): 添加事实
assert_fact('parent', 'Qianlong','Jiaqing')
# 相当于 '+ parent('Qianlong', 'Jiaqing') '

# ask: 通过字符串执行指令，并把answer作为元组返回
print(ask("ancestor('Kangxi',Y)")) 

# assert_fact(string,不限量任意参数): 删除事实
retract_fact('parent', 'Qianlong','Jiaqing')
# 相当于 '- parent('Qianlong', 'Jiaqing')
print(ask("ancestor('Kangxi',Y)")) 

{('Qianlong',), ('Jiaqing',), ('Yongzheng',)}
{('Qianlong',), ('Yongzheng',)}


有了这些操作，我们就可以想一些办法来处理更通用的数据格式了，比如说Excel表格(这里使用pandas的DataFrame演示一个简单例子)：

In [2]:
import pandas as pd
pyDatalog.clear()   #清空前面学到的事实

# 简单的表示关系的数据表
data = pd.DataFrame({"object":['Kangxi','Yongzheng','Qianlong'],
                     "relation":['parent','parent','parent'],
                     "subject":['Yongzheng','Qianlong','Jiaqing']})
print(data)

      object relation    subject
0     Kangxi   parent  Yongzheng
1  Yongzheng   parent   Qianlong
2   Qianlong   parent    Jiaqing


数据表转为Datalog语句：

In [3]:
data_str = ""
for line_id,line in data.iterrows():
    obj,rela,subj = line["object"],line["relation"],line["subject"]
    data_str += "+ %s('%s','%s')\n" % (rela,obj,subj)
print(data_str)
load(data_str)

+ parent('Kangxi','Yongzheng')
+ parent('Yongzheng','Qianlong')
+ parent('Qianlong','Jiaqing')



数据表转为Datalog语句：

In [5]:
data_str = ""

for line_id,line in data.iterrows():
    obj,rela,subj = line["object"],line["relation"],line["subject"]
    data_str += "+ %s('%s','%s')\n" % (rela,obj,subj)

print(data_str)
load(data_str)

+ parent('Kangxi','Yongzheng')
+ parent('Yongzheng','Qianlong')
+ parent('Qianlong','Jiaqing')



数据表转为Datalog语句：

In [6]:
rules = """
    ancestor(X,Y) <= parent(X,Y)
    ancestor(X,Y) <= parent(X,Z) & ancestor(Z,Y)
    """

load(rules)
print(ask("ancestor(X,Y)")) 

{('Kangxi', 'Qianlong'), ('Kangxi', 'Yongzheng'), ('Yongzheng', 'Jiaqing'), ('Qianlong', 'Jiaqing'), ('Kangxi', 'Jiaqing'), ('Yongzheng', 'Qianlong')}
